<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Problem-1" data-toc-modified-id="Problem-1-1">Problem 1</a></span></li><li><span><a href="#Problem-2" data-toc-modified-id="Problem-2-2">Problem 2</a></span></li></ul></div>

In [3]:
import pulp as pl

from pulp import LpProblem
from pulp import lpSum
from pulp import LpVariable

## Problem 1

In [20]:
model_1 = LpProblem(name="problem-1", sense=pl.LpMinimize)

# Decision variables
variables = {}
for i in range(1, 16):
    for j in range(1, 4):
        variables[f"x_{i}_{j}"] = LpVariable(name=f"x_{i}_{j}", cat="Binary")
z = LpVariable(name="max_makespan", lowBound=0)

# Objective function
model_1 += z

# Constraints
# any job is only on one machine
for i in range(1, 16):
    model_1 += (
        lpSum([variables[f"x_{i}_{j}"] for j in range(1, 4)]) == 1, 
        f"job_{i}_on_one_machine",
    )

# max makespan
processing_times = [7, 4, 6, 9, 12, 8, 10, 11, 8, 7, 6, 8, 15, 14, 3]
for j in range(1, 4):
    model_1 += (
        z >= lpSum([processing_times[i] * variables[f"x_{i + 1}_{j}"] for i in range(15)]), 
        f"Makespan-for-machine-{j}",
    )

# exclusivity 
exclusivity = [(2, 5, 8), (6, 9), (7, 10), (11, 15)]
for e in exclusivity:
    for j in range(1, 4):
        model_1 += (
            lpSum([variables[f"x_{ee}_{j}"] for ee in e]) <= 1,
            f"jobs_{'_'.join((map(str, e)))}_not_on_machine_{j}"
        )

status = model_1.solve(pl.PULP_CBC_CMD(msg=0))
if status == 1:
    print(model_1.objective.value())

43.0


## Problem 2

In [23]:
distance = [
    [0, 3, 4, 6, 8, 9, 8, 10],
    [3, 0, 5, 4, 8, 6, 12, 9],
    [4, 5, 0, 2, 2, 3, 5, 7],
    [6, 4, 2, 0, 3, 2, 5, 4],
    [8, 8, 2, 3, 0, 2, 2, 4],
    [9, 6, 3, 2, 2, 0, 3, 2],
    [8, 12, 5, 5, 2, 3, 0, 2],
    [10, 9, 7, 4, 4, 2, 2, 0],
]

population = [40, 30, 35, 20, 15, 50, 45, 60]

n = len(distance)
m = 2

# Model
model_2 = LpProblem(name="quiz-2", sense=pl.LpMinimize)

# Decision variable
variables = {f"x_{i}": LpVariable(name=f"x_{i}", cat="Binary") for i in range(n)}

# Constraints
# m ambulances in total
model_2 += (lpSum([variables[f"x_{i}"] for i in range(n)]) == m, f"{m}-ambulances-in-total")